In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import openpyxl
import warnings
warnings.filterwarnings("ignore")

# import http.client
# http.client._MAXHEADERS = 1000
pd.set_option('display.max_columns', None)

Data download works on 01-11-2023, all classes may change over time

In [ ]:
#number of pages
main_link = 'https://www.otomoto.pl/osobowe/audi/a5'
main_link_r = requests.get(main_link)
html_main_link = BeautifulSoup(main_link_r.text, 'html.parser')
max_page_nbr = int(html_main_link.find_all('a',{'class':'ooa-xdlax9 e1f09v7o0'})[-1].get_text()) #page navigation at the bottom

In [ ]:
#list of pages links
base_link = 'https://www.otomoto.pl/osobowe/audi/a5?page={}'
all_pages_links = [base_link.format(i+1) for i in range(max_page_nbr)]

In [ ]:
#all links from one page
def links_from_page(input_page_link):
    r_get = requests.get(input_page_link)
    html = BeautifulSoup(r_get.text, 'html.parser')
    page_links_pre = html.find_all('h1',{'class':'ev7e6t89 ooa-1xvnx1e er34gjf0'}) #direct link to single ad
    page_ads_links = [lnk.find('a')['href'] for lnk in page_links_pre]

    return page_ads_links

In [ ]:
#all links from all pages
all_ad_links = []
broken_page_link_counter = 0
for page_link in all_pages_links: #place to put limit on test
    tmp_link = links_from_page(page_link)
    counter_page = 0
    while tmp_link == [] and counter_page < 20: #second loop due to occasional data download errors
        tmp_link = links_from_page(page_link)
        counter_page += 1
        time.sleep(2)
    if counter_page < 20:
        all_ad_links += tmp_link
    else:
        broken_page_link_counter += 1
        
    time.sleep(1)

In [ ]:
#one feature from ad
def one_feature(A,B,html_temp):
    try:
        feature_name = html_temp.find(A, {'class':B}).get_text()
    except:
        feature_name = ''
    return feature_name

In [ ]:
#all data from one ad
def extract_data_from_one_ad_link(ad_link):
    r_temp = requests.get(ad_link)
    html_temp = BeautifulSoup(r_temp.text, 'html.parser')

    #title
    title_temp = one_feature('h3','offer-title big-text e1aiyq9b1 ooa-ebtemw er34gjf0',html_temp) #class with title, top right corner
    title_temp = title_temp.strip()

    #price
    price_temp = one_feature('h3','offer-price__number esicnpr5 ooa-17vk29r er34gjf0',html_temp) #class with price, top right corner
    price_temp = price_temp.replace('PLN','').strip()

    features_dict= {}
    features_dict['Tytuł'] = title_temp
    features_dict['Cena'] = price_temp
    features_dict['Link'] = ad_link

    #other features
    features = html_temp.find_all('div', {'data-testid':'advert-details-item'}) #class above every feature inside single ad
    for feature in features:
        feature_name = feature.find('p', {'class':'e1iqsx43 ooa-1h25pzj'}).get_text() #class with name of the single feature
        try:
            feature_value = feature.find('a', {'class':'e1cl5iuz0 ooa-189feh2'}) #class for feature value presented as links
            feature_value = feature_value.get_text().strip()
        except:
            try:
                feature_value = feature.find('p', {'class':'e1cl5iuz1 ooa-ywgrlx er34gjf0'}) #class for feature value presented as text
                feature_value = feature_value.get_text().strip()
            except:
                feature_value = None
        features_dict[feature_name] = feature_value
    
    
    return features_dict

In [ ]:
#data from all the ads
result_df = pd.DataFrame(columns = ['Tytuł','Cena','Kraj pochodzenia', 'Leasing', 'Pokaż oferty z numerem VIN', 'Wersja', 'Numer rejestracyjny pojazdu', 'Rodzaj koloru', 'lub do (przebieg km)', 'Spalanie W Mieście', 'Kolor', 'Zarejestrowany w Polsce', 'Tuning', 'Bezwypadkowy', 'Moc',
'Kategoria', 'Liczba drzwi', 'Tytuł', 'Marka pojazdu', 'Spalanie Poza Miastem', 'Oferta od', 'Faktura VAT', 'Emisja CO2', 'Spalanie W Cyklu Mieszanym', 'Przebieg', ' Pierwszy właściciel (od nowości)', 'Liczba miejsc', 'Data pierwszej rejestracji w historii pojazdu', 
'Rok produkcji', 'Rodzaj paliwa', 'Pojemność skokowa', 'Miesięczna rata', 'Generacja', 'Stan', 'Skrzynia biegów', 'Opłata początkowa', 'VAT marża', 'Filtr cząstek stałych', 'Serwisowany w ASO', 'Model pojazdu', 'Okres gwarancji producenta', 'Wartość wykupu', 
'Uszkodzony', 'Ma numer rejestracyjny', 'Hak', 'Typ nadwozia', 'Możliwość finansowania', 'Napęd', 'Liczba pozostałych rat'])

broken_link_counter = 0
for ad in all_ad_links:
    result_df_tmp = pd.DataFrame(columns = ['Tytuł','Cena','Kraj pochodzenia', 'Leasing', 'Pokaż oferty z numerem VIN', 'Wersja', 'Numer rejestracyjny pojazdu', 'Rodzaj koloru', 'lub do (przebieg km)', 'Spalanie W Mieście', 'Kolor', 'Zarejestrowany w Polsce', 'Tuning', 'Bezwypadkowy', 'Moc',
    'Kategoria', 'Liczba drzwi', 'Tytuł', 'Marka pojazdu', 'Spalanie Poza Miastem', 'Oferta od', 'Faktura VAT', 'Emisja CO2', 'Spalanie W Cyklu Mieszanym', 'Przebieg', ' Pierwszy właściciel (od nowości)', 'Liczba miejsc', 'Data pierwszej rejestracji w historii pojazdu', 
    'Rok produkcji', 'Rodzaj paliwa', 'Pojemność skokowa', 'Miesięczna rata', 'Generacja', 'Stan', 'Skrzynia biegów', 'Opłata początkowa', 'VAT marża', 'Filtr cząstek stałych', 'Serwisowany w ASO', 'Model pojazdu', 'Okres gwarancji producenta', 'Wartość wykupu', 
    'Uszkodzony', 'Ma numer rejestracyjny', 'Hak', 'Typ nadwozia', 'Możliwość finansowania', 'Napęd', 'Liczba pozostałych rat'])
    result_df_tmp.loc[0] = None
    
    tmp_dict = extract_data_from_one_ad_link(ad)
    counter_ad = 0
    while len(tmp_dict) < 3 and counter_ad < 20: #second loop due to occasional data download errors
        tmp_dict = extract_data_from_one_ad_link(ad)
        counter_ad += 1
        time.sleep(2)

    if counter_ad < 20:
        for col in result_df_tmp.columns:
            try:
                result_df_tmp[col] = tmp_dict[col]
            except:
                result_df_tmp[col] = None
        result_df = pd.concat([result_df, result_df_tmp], ignore_index=True)
        if len(result_df) %100 == 0:
            print(len(result_df))
    else:
        broken_link_counter += 1

    time.sleep(1)

In [ ]:
test = len(all_ad_links) == len(result_df) + broken_link_counter

if test == True:
    print('OK')
else:
    print('broken_page_link_counter:',broken_page_link_counter)
    print('broken_link_counter:',broken_link_counter)

In [ ]:
result_df.to_excel('otomoto_v2.xlsx', index = False)